In [2]:
import numpy as np
import pandas as pd
import tkinter as tk
from tkinter import ttk

# Tạo ứng dụng Tkinter
app = tk.Tk()
app.title("Đánh giá trường đại học")

tree_columns = ("Tên trường", "Điểm tương tự")
tree = ttk.Treeview(app, columns=tree_columns, show="headings")
df = pd.read_csv("./Data_NguyenVong.csv")

def evaluate_universities():
    # Lấy dữ liệu từ người dùng
    B1 = float(entry_fee.get())
    B2 = float(entry_score.get())    
    location_mapping = {'Miền bắc': 1, 'Miền trung': 2, 'Miền nam': 3}
    B3 = location_mapping.get(combo_location.get(), 0)  # Sử dụng get để tránh lỗi khi giá trị không tồn tại
    
    C_list = []
    dia_diem_dict = {'Miền bắc': 1, 'Miền trung': 2, 'Miền nam': 3}

    for i in range(len(df)):
        A1 = df['Học phí'][i]
        A2 = df['Đánh giá'][i]
        A3 = dia_diem_dict[df['Địa điểm'][i]]
        A4 = df['Điểm dự kiến'][i]
        A5 = df['Điểm chuẩn năm trước'][i]
        A6 = df['Chỉ tiêu'][i]

        # Sự phù hợp về học phí
        if B1 >= 2 * A1:
            C1 = 1
        elif B1 <= A1:
            C1 = 0
        else:
            C1 = (B1 - A1) / A1

        # Đánh giá trường
        C2 = A2
        # Sự phù hợp về địa điểm
        if abs(A3-B3)==0:
            C3=0
        elif abs(A3-B3)==1:
            C3=0.5
        else:
            C3=0
        # Sự phù hợp về điểm thi
        C4 = (B2 - (A4 - abs(A5 - A4))) / (2 * abs(A5 - A4))
        C4 = max(0, min(1, C4))  # Giới hạn C4 trong khoảng [0, 1]

        # Sự phù hợp về chỉ tiêu
        C5 = A6

        # Tạo một mảng để lưu trữ các giá trị của bảng quyết định
        C = np.array([C1, C2, C3, C4, C5])

        # Thêm mảng vào danh sách
        C_list.append(C)

    # Chuyển đổi danh sách thành một ma trận
    C_matrix = np.array(C_list)

    # Chuẩn hóa ma trận theo cột
    C_matrix = C_matrix / np.sqrt(np.sum(C_matrix ** 2, axis=0))

    # Giả sử các trọng số của các thuộc tính là như sau
    W = np.array([0.2, 0.2, 0.2, 0.2, 0.2])  # Trọng số

    # Nhân ma trận với trọng số
    C_matrix = C_matrix * W

    # Tính phương án tối ưu và phương án tệ nhất
    A_opt = np.max(C_matrix, axis=0)  # Phương án tối ưu
    A_worst = np.min(C_matrix, axis=0)  # Phương án tệ nhất

    # Tính khoảng cách từ mỗi phương án đến phương án tối ưu và phương án tệ nhất
    D_opt = np.sqrt(np.sum((C_matrix - A_opt) ** 2, axis=1))  # Khoảng cách đến phương án tối ưu
    D_worst = np.sqrt(np.sum((C_matrix - A_worst) ** 2, axis=1))  # Khoảng cách đến phương án tệ nhất

    # Tính độ tương tự với phương án tối ưu và phương án tệ nhất
    S = D_worst / (D_opt + D_worst)  # Độ tương tự

    # Chọn phương án có độ tương tự cao nhất là phương án tốt nhất cho bài toán
    best_index = np.argmax(S)  # Chỉ số của phần tử lớn nhất trong mảng

    # Hiển thị kết quả trên giao diện
    result_label.config(text=f"Phương án tốt nhất cho thí sinh là: {df.iloc[best_index]['Tên trường']}")
    result_text.delete(1.0, tk.END)
    result_text.insert(tk.END, df.iloc[best_index])

def reset_inputs():
    entry_fee.delete(0, tk.END)
    entry_score.delete(0, tk.END)
    combo_location.set('')  # Modified line
    result_label.config(text="")
    result_text.delete(1.0, tk.END)


# Tạo và thiết lập các widget
label_fee = tk.Label(app, text="Học phí:")
entry_fee = tk.Entry(app)

label_score = tk.Label(app, text="Điểm thi:")
entry_score = tk.Entry(app)

label_location = tk.Label(app, text="Địa điểm:")
locations = ['Miền bắc', 'Miền trung', 'Miền nam']
combo_location = ttk.Combobox(app, values=locations)
combo_location.set('')  # Default selection is empty

btn_evaluate = tk.Button(app, text="Đánh giá", command=evaluate_universities)
btn_reset = tk.Button(app, text="Nhập lại", command=reset_inputs)

result_label = tk.Label(app, text="")
result_text = tk.Text(app, wrap=tk.WORD, height=10, width=50)

# Đặt các widget vào giao diện
label_fee.grid(row=0, column=0, padx=10, pady=10)
entry_fee.grid(row=0, column=1, padx=10, pady=10)

label_score.grid(row=1, column=0, padx=10, pady=10)
entry_score.grid(row=1, column=1, padx=10, pady=10)

label_location.grid(row=2, column=0, padx=10, pady=10)
combo_location.grid(row=2, column=1, padx=10, pady=10)

btn_evaluate.grid(row=3, column=0, pady=10)
btn_reset.grid(row=3, column=1, pady=10)

result_label.grid(row=4, column=0, columnspan=2, pady=10)
result_text.grid(row=5, column=0, columnspan=2, pady=10)

# Chạy ứng dụng
app.mainloop()


In [3]:
df

,Tên trường,Chuyên ngành,Học phí,Đánh giá,Địa điểm,Điểm dự kiến,Điểm chuẩn năm trước,Chỉ tiêu
0,Đại học Bách Khoa,Khoa Kỹ thuật,20000000,1.0,Miền bắc,28.0,27.5,200
1,Đại học Kinh doanh và Quản trị doanh nY dược v...,Khoa Kinh doanh,25000000,1.0,Miền bắc,29.5,28.0,150
2,Đại học Đại học Nghệ thuật và Truyền thông,Khoa Nghệ thuật,18000000,0.5,Miền nam,26.0,25.5,180
3,Đại học Y dược,Khoa Y học,30000000,1.0,Miền bắc,27.5,27.0,120
4,Đại học Xã hội học,Khoa Khoa học Xã hội,22000000,0.5,Miền nam,25.5,25.0,160
5,Đại học Khoa học Tự nhiên,Khoa Khoa học Tự nhiên,26000000,1.0,Miền bắc,28.8,28.2,140
6,Đại học Luật học,Khoa Luật,28000000,1.0,Miền bắc,27.2,26.8,100
7,Đại học Y học và Dược học,Khoa Y học,32000000,1.0,Miền bắc,29.2,28.5,130
8,Đại học Kỹ thuật Công nghiệp,Khoa Kỹ thuật,19000000,1.0,Miền bắc,27.0,26.5,180
9,Đại học Xã hội học,Khoa Kinh doanh,24000000,0.5,Miền nam,29.0,28.3,150
